* selenium 안쓰고 좋아요 크롤링 하려 했는데 안되길래 어쩔 수 없이 사용. 
* 문제 : 속도가 많이 느려짐.....약 5~6배?

In [3]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

# 멜론 TOP 100 차트 페이지 URL
url = 'https://www.melon.com/chart/index.htm'

# Selenium 웹 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # 헤드리스 모드(브라우저 창을 열지 않음)
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# 웹 드라이버 실행
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get(url)

# 페이지 로드 대기
time.sleep(3)

# 곡 ID, 제목, 아티스트 추출
song_ids = []
song_titles = []
song_artists = []

for song_tag in driver.find_elements(By.CSS_SELECTOR, 'tr.lst50, tr.lst100'):
    song_id = song_tag.get_attribute('data-song-no')
    if song_id:
        title_tag = song_tag.find_element(By.CSS_SELECTOR, 'div.rank01 a')
        artist_tags = song_tag.find_elements(By.CSS_SELECTOR, 'div.rank02 a')
        song_ids.append(song_id)
        song_titles.append(title_tag.text)
        song_artists.append(', '.join([artist.text for artist in artist_tags]))

driver.quit()

# 곡 가사 및 좋아요 수 추출 함수
def get_song_details(song_id):
    local_driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    try:
        song_url = f'https://www.melon.com/song/detail.htm?songId={song_id}'
        local_driver.get(song_url)
        time.sleep(1)  # 페이지 로드 대기
        
        # 가사 추출
        lyrics_tag = local_driver.find_element(By.CSS_SELECTOR, 'div.lyric')
        lyrics = lyrics_tag.text.replace(',', '') if lyrics_tag else "No lyrics found"

        # 좋아요 수 추출
        like_button = local_driver.find_element(By.CSS_SELECTOR, 'button.like')
        like_tag = like_button.find_element(By.CSS_SELECTOR, 'span.cnt')
        likes = like_tag.text.replace('총건수', '').strip().replace(',', '') if like_tag else '0'
        
        return song_id, lyrics, likes
    except Exception as e:
        print(f"Error fetching details for song ID {song_id}: {e}")
        return song_id, "No lyrics found", "0"
    finally:
        local_driver.quit()

# 병렬처리를 위해 ThreadPoolExecutor 사용
with ThreadPoolExecutor(max_workers=10) as executor:  # 병렬 처리를 위한 스레드풀 생성, max_workers는 병렬화할 작업 수
    results = list(executor.map(get_song_details, song_ids))

# 결과 처리
lyrics_dict = {song_id: lyrics for song_id, lyrics, _ in results}
likes_dict = {song_id: likes for song_id, _, likes in results}
lyrics_list = [lyrics_dict[song_id] for song_id in song_ids]
likes_list = [likes_dict[song_id] for song_id in song_ids]

# 데이터프레임 생성
data = {'Song ID': song_ids, 'Title': song_titles, 'Artist': song_artists, 'Lyrics': lyrics_list, 'Likes': likes_list}
df = pd.DataFrame(data)

# CSV 파일로 저장
df.to_csv('melon_top_100(add LIKES).csv', index=False, encoding='utf-8-sig')

print("CSV 파일이 저장되었습니다.")


CSV 파일이 저장되었습니다.
